In [29]:
import pandas as pd
import numpy as np

In [30]:
df_train=pd.read_csv('../input/train.csv.zip')

In [31]:
df_train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0



__First__ predictor is the matching words coefficient


In [32]:
from nltk.corpus import stopwords

stops = set(stopwords.words("english"))

def matching_words_coeff(row):
    question1_dic={}
    question2_dic={}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            question1_dic[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
             question2_dic[word] = 1
    if len(question1_dic) == 0 or len(question2_dic) == 0:
        return 0
    shared_words_in_q1 = [w for w in question1_dic.keys() if w in question2_dic]
    shared_words_in_q2 = [w for w in question2_dic.keys() if w in question1_dic]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))/(len(question1_dic) + len(question2_dic))
    return R
    

In [33]:
train_word_match_coeff = df_train.apply(matching_words_coeff, axis=1)
train_word_match_coeff.head()


0    0.727273
1    0.307692
2    0.363636
3    0.000000
4    0.000000
dtype: float64


<!-- __Second__ predictor will be cosine similarity -->


In [ ]:
# import spacy
# from sklearn.metrics.pairwise import cosine_similarity
# nlp=spacy.load("en_core_web_md")
# question1_vectors=np.array([nlp(question).vector for question in df_train.question1])
# question2_vectors=np.array([nlp(question).vector for question in df_train.question2])

# print(question1_vectors.shape)

In [34]:
x_train = pd.DataFrame()
x_train['word_mathc_coeff'] = train_word_match_coeff
y_train=df_train.is_duplicate


In [36]:
# splitting for validation
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [44]:
import xgboost as xgb

# Set our parameters for xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 4

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=50, verbose_eval=10)


Preparing test data


In [46]:
df_test=pd.read_csv('../input/test.csv.zip')
df_test.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [48]:
test_coeff=df_test.apply(matching_words_coeff, axis=1)
x_test = pd.DataFrame()
x_test['word_mathc_coeff'] = test_coeff

In [49]:
d_test = xgb.DMatrix(x_test)
p_test = bst.predict(d_test)
#
sub = pd.DataFrame()
sub['test_id'] = df_test['test_id']
sub['is_duplicate'] = p_test
sub.to_csv('xgboost.csv', index=False)